### 01 packages

In [8]:
##########################################################################################
##########################################################################################

import networkx as nx

from shapely.geometry import Point,Polygon

import math

import random

##########################################################################################
##########################################################################################

import numpy as np

import pandas as pd

##########################################################################################
##########################################################################################

import matplotlib

import matplotlib.pyplot as plt

from matplotlib.ticker import MultipleLocator, FormatStrFormatter

##########################################################################################
##########################################################################################

from itertools import combinations

##########################################################################################
##########################################################################################

import copy

import os


### 02 interior parameters

In [20]:
##########################################################################################
##########################################################################################

Base_speed=int(10000/60) # 10 KM per hour

##########################################################################################
##########################################################################################

alpha=1

speed=alpha*Base_speed

##########################################################################################
##########################################################################################

batch_interval=1

##########################################################################################
##########################################################################################

instance='01'

##########################################################################################
##########################################################################################

lambda_value = 20

pattern='uniform'#'centralized','uniform'

##########################################################################################
##########################################################################################

save_path="./03result/01Direct_Delivery/"+pattern+'_'+str(lambda_value)

if not os.path.exists(save_path):
    
    os.makedirs(save_path)

##########################################################################################
##########################################################################################

Nodes=np.load("./01network/"+instance+"/Nodes.npy")

Edges=np.load("./01network/"+instance+"/Edges.npy")

##########################################################################################
##########################################################################################

Road_Graph = nx.Graph()

for node in Nodes:
    
    node=tuple(node)
        
    Road_Graph.add_node(node)

for edge in Edges:
    
    node1=tuple(edge[0])
    
    node2=tuple(edge[1])
    
    Road_Graph.add_edge(node1, node2, weight=Point(node1).distance(Point(node2)))


In [21]:
def total_travel_distance(trajectory):
    
    total_distance = 0.0
    
    # Iterate through the trajectory and calculate the distance for each consecutive pair of coordinates
    
    for i in range(len(trajectory) - 1):
        
        total_distance += nx.shortest_path_length(Road_Graph, source=trajectory[i], target=trajectory[i + 1], weight='weight') 
    
    return total_distance

### 04 Parcel-Courier assignment by EDD

In [22]:
##########################################################################################
##########################################################################################

parcel_df=pd.read_csv('./02data/01/instance_'+str(lambda_value)+'_'+pattern+'.csv')

parcel_df=parcel_df.drop(columns=['Unnamed: 0'])

##########################################################################################
##########################################################################################

parcel_df['pick_up']=parcel_df.apply(lambda x:tuple([round(float(y),2) for y in x.pick_up.replace('(','').replace(')','').split(',')]),axis=1)

parcel_df['drop_off']=parcel_df.apply(lambda x:tuple([round(float(y),2) for y in x.drop_off.replace('(','').replace(')','').split(',')]),axis=1)

##########################################################################################
##########################################################################################

parcel_df['courier_id']='None'

parcel_df['pick_up_time']=-float('inf')

parcel_df['drop_off_time']=-float('inf')

##########################################################################################
##########################################################################################

courier_df=pd.read_csv('./02data/'+instance+'/courier_instance.csv')

courier_df=courier_df.drop(columns=['Unnamed: 0'])

##########################################################################################
##########################################################################################

courier_schedules={}

for idx,row in courier_df.iterrows():
    
    courier_schedules[row.courier_id]={i:(row.x,row.y) for i in range(1,batch_interval+1,1)}
    
##########################################################################################
##########################################################################################

t=copy.copy(batch_interval)

##########################################################################################
##########################################################################################


dynamic_completions={}

dynamic_distances={}

while True:
    
    ##########################################################################################
    ##########################################################################################
    
    # 01 collect parcels
    
    collected_parcel=copy.copy(parcel_df.loc[(parcel_df.t_a<=t)&(parcel_df.courier_id=='None')])
    
    ##########################################################################################
    ##########################################################################################
    
    # select parcel with the earliest due time
    
    collected_parcel=collected_parcel.sort_values(by='t_dl', ascending=True)
    
    collected_parcel=collected_parcel.reset_index(drop=True)
    
    ##########################################################################################
    ##########################################################################################
    
    while not collected_parcel.empty:
        
        ##########################################################################################
        ##########################################################################################
        
        # select the parcel
    
        select_parcel=collected_parcel.iloc[0]
        
        parcel_id=select_parcel.parcel_id
        
        pick_up_point=select_parcel.pick_up
        
        drop_off_point=select_parcel.drop_off
        
        ##########################################################################################
        ##########################################################################################
        
        pick_up_times={}
        
        completion_times={}
        
        for courier_id,schedule in courier_schedules.items():
            
            courier_realse_time=list(schedule.keys())[-1]
            
            courier_realse_point=tuple(list(schedule.values())[-1])
            
            pick_up_distance=nx.shortest_path_length(Road_Graph, source=courier_realse_point, target=pick_up_point, weight='weight')
            
            drop_off_distance=nx.shortest_path_length(Road_Graph, source=pick_up_point, target=drop_off_point, weight='weight')
            
            pick_up_time=int(courier_realse_time+pick_up_distance/speed)
            
            completion_time=int(pick_up_time+drop_off_distance/speed)
            
            pick_up_times[courier_id]=pick_up_time
            
            completion_times[courier_id]=completion_time
            
        ##########################################################################################
        ##########################################################################################
            
        select_courier_id=min(completion_times, key=completion_times.get)
        
        select_courier_realse_time=list(courier_schedules[select_courier_id].keys())[-1]
        
        select_courier_realse_point=tuple(list(courier_schedules[select_courier_id].values())[-1])
        
        parcel_df.loc[(parcel_df['parcel_id']==parcel_id),'courier_id']=select_courier_id
        
        ##########################################################################################
        ##########################################################################################
        
        # update the schedule of couriers
        
        first_schedule=nx.shortest_path(Road_Graph, source=select_courier_realse_point, target=pick_up_point, weight='weight')
        
        second_schedule=nx.shortest_path(Road_Graph, source=pick_up_point, target=drop_off_point, weight='weight')
        
        for i in range(1,len(first_schedule),1):
            
            distance=nx.shortest_path_length(Road_Graph, source=first_schedule[i-1], target=first_schedule[i], weight='weight')
            
            select_courier_realse_time=select_courier_realse_time+int(distance/speed)
            
            courier_schedules[select_courier_id][select_courier_realse_time]=(first_schedule[i][0],first_schedule[i][1])
            
        parcel_df.loc[(parcel_df['parcel_id']==parcel_id),'pick_up_time']=select_courier_realse_time
            
        for j in range(1,len(second_schedule),1):
            
            distance=nx.shortest_path_length(Road_Graph, source=second_schedule[j-1], target=second_schedule[j], weight='weight')
            
            select_courier_realse_time=select_courier_realse_time+int(distance/speed)
            
            courier_schedules[select_courier_id][select_courier_realse_time]=(second_schedule[j][0],second_schedule[j][1])
            
        parcel_df.loc[(parcel_df['parcel_id']==parcel_id),'drop_off_time']=select_courier_realse_time
            
        ##########################################################################################
        ##########################################################################################
        
        # remove the selected parcel from the unassigned parcels
        
        collected_parcel = collected_parcel.drop(collected_parcel.index[0])
        
        collected_parcel = collected_parcel.reset_index(drop=True)
        
        
    ##########################################################################################
    ##########################################################################################
    
    # update the batch time
    
    for courier_id,schedules in courier_schedules.items():
            
        courier_realse_time=list(schedules.keys())[-1]

        courier_realse_point=tuple(list(schedules.values())[-1])

        if courier_realse_time==t:

            for i in range(1,batch_interval+1,1):

                 courier_schedules[courier_id][t+i]=courier_realse_point
                    
    ##########################################################################################
    ##########################################################################################
    
    t=int(t+batch_interval)
    
    if t%10==0:
            
        ttdis=0.0

        for schedules in courier_schedules.values():

            trajectory=list([v for k,v in schedules.items() if k<=t])

            ttdis+=total_travel_distance(trajectory)

        print("*"*100)

        print("Minute is ",t,", Number of completions is ",parcel_df.loc[(parcel_df.drop_off_time>=0)&(t>parcel_df.drop_off_time)].shape[0],\
              ", Total travel distance(KM) is ",round(ttdis/1000.0,2))
        
        dynamic_completions[t]=parcel_df.loc[(parcel_df.drop_off_time>=0)&(t>parcel_df.drop_off_time)].shape[0]
            
        dynamic_distances[t]=round(ttdis/1000.0,2)
    

    if parcel_df.loc[(parcel_df.courier_id=='None')].empty and t>parcel_df.drop_off_time.max():
    
        break


****************************************************************************************************
Minute is  10 , Number of completions is  0 , Total travel distance(KM) is  28.0
****************************************************************************************************
Minute is  20 , Number of completions is  8 , Total travel distance(KM) is  84.0
****************************************************************************************************
Minute is  30 , Number of completions is  12 , Total travel distance(KM) is  112.0
****************************************************************************************************
Minute is  40 , Number of completions is  21 , Total travel distance(KM) is  168.0
****************************************************************************************************
Minute is  50 , Number of completions is  27 , Total travel distance(KM) is  224.0
************************************************************************************

****************************************************************************************************
Minute is  480 , Number of completions is  340 , Total travel distance(KM) is  2212.0
****************************************************************************************************
Minute is  490 , Number of completions is  350 , Total travel distance(KM) is  2268.0
****************************************************************************************************
Minute is  500 , Number of completions is  363 , Total travel distance(KM) is  2324.0
****************************************************************************************************
Minute is  510 , Number of completions is  365 , Total travel distance(KM) is  2352.0
****************************************************************************************************
Minute is  520 , Number of completions is  372 , Total travel distance(KM) is  2408.0
*****************************************************************

****************************************************************************************************
Minute is  940 , Number of completions is  694 , Total travel distance(KM) is  4368.0
****************************************************************************************************
Minute is  950 , Number of completions is  702 , Total travel distance(KM) is  4424.0
****************************************************************************************************
Minute is  960 , Number of completions is  710 , Total travel distance(KM) is  4452.0
****************************************************************************************************
Minute is  970 , Number of completions is  718 , Total travel distance(KM) is  4508.0
****************************************************************************************************
Minute is  980 , Number of completions is  727 , Total travel distance(KM) is  4564.0
*****************************************************************

### 05 save results

In [23]:
##########################################################################################
##########################################################################################

# 01 save parcel

parcel_df.to_csv(save_path+"/parcel_df_"+str(alpha)+"_"+str(batch_interval)+".csv")

##########################################################################################
##########################################################################################

def generate_trajectory(origin, destination):
    
    # Calculate the total distance
    
    distance = Point(destination).distance(Point(origin))
    
    # Calculate total time
    total_time = distance / speed
    
    # Calculate the number of intervals
    intervals = int(np.floor(total_time))
    
    # Create an array to hold the trajectory
    trajectory = np.zeros((intervals+1, 2))  # Assuming 2D coordinates
    
    for i in range(intervals+1):
        # Interpolate the position for each time interval
        alpha = i / intervals
        trajectory[i] = np.array(origin) * (1 - alpha) + np.array(destination) * alpha

    return trajectory

##########################################################################################
##########################################################################################

# 02 complete trajectory

for courier_id,schedules in courier_schedules.items():
    
    completed_schedules=copy.copy(schedules)
    
    stamps=list(schedules.keys())
    
    nodes=list(schedules.values())
    
    for i in range(1,len(stamps),1):
        
        pre_t,t=stamps[i-1],stamps[i]
        
        pre_node,node=nodes[i-1],nodes[i]
        
        if t>pre_t+1:
            
            trajectory=generate_trajectory(pre_node, node)
            
            for j in range(1,len(trajectory)-1,1):
                
                insert_node=tuple([round(x,2) for x in trajectory[j]])
                
                completed_schedules[int(pre_t+j)]=insert_node
                
    completed_schedules = dict(sorted(completed_schedules.items()))
    
    courier_schedules[courier_id]=completed_schedules
            
##########################################################################################
##########################################################################################

np.save(save_path+"/courier_schedules_"+str(alpha)+"_"+str(batch_interval),courier_schedules)

np.save(save_path+"/dynamic_completions_"+str(alpha)+"_"+str(batch_interval),dynamic_completions)

np.save(save_path+"/dynamic_distances_"+str(alpha)+"_"+str(batch_interval),dynamic_distances)   



In [24]:
parcel_df

,parcel_id,t_a,pick_up,drop_off,h1,h2,shortest_distance,t_dl,courier_id,pick_up_time,drop_off_time
0,p1,0,"(1000.0, 3464.1)","(2500.0, 866.03)",H19,H8,2999,127,c19,13.0,31.0
1,p2,0,"(500.0, 2598.08)","(-1500.0, 866.03)",H18,H6,2999,133,c6,13.0,31.0
2,p3,0,"(-1000.0, 1732.05)","(3500.0, -2598.08)",H6,H10,7000,336,c6,37.0,79.0
3,p4,0,"(3000.0, 1732.05)","(-2000.0, -1732.05)",H8,H5,6999,302,c12,49.0,91.0
4,p5,0,"(500.0, -2598.08)","(-3500.0, -866.03)",H12,H14,4999,255,c2,19.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...
995,p996,49,"(-4000.0, -1732.05)","(1000.0, 0.0)",H14,H2,5999,290,c15,1315.0,1351.0
996,p997,49,"(500.0, 866.03)","(2000.0, -0.0)",H1,H2,2000,126,c7,1297.0,1309.0
997,p998,49,"(1000.0, 3464.1)","(-4000.0, -1732.05)",H18,H14,7999,505,c25,1321.0,1369.0
998,p999,49,"(500.0, 866.03)","(0.0, -3464.1)",H7,H12,5000,223,c23,1303.0,1333.0
